<a href="https://colab.research.google.com/github/karasevdy/pet_projects/blob/main/Gorky_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Графики для Горького

In [2]:
import random
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import time as t
import re
from tqdm import tqdm_notebook
from tqdm import notebook
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

!pip install bokeh
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.transform import factor_cmap, factor_mark

!pip install -U plotly
import plotly.express as px

In [3]:
df = pd.read_csv('gorky_join_4243_lemms_df.scv')

In [4]:
m = pd.read_csv('gorky_join_4243_embeddings_YaGPT')

In [5]:
df = df.drop('Unnamed: 0', axis=1)
m = m.drop('Unnamed: 0', axis=1)

In [6]:
features = m.values

In [7]:
print(features.shape)

(4243, 1024)


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4243 entries, 0 to 4242
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   url          4243 non-null   object
 1   author       4243 non-null   object
 2   publ_date    4243 non-null   object
 3   title        4243 non-null   object
 4   subtitle     4243 non-null   object
 5   description  4243 non-null   object
 6   text         4243 non-null   object
 7   month_year   4243 non-null   object
 8   title_len    4243 non-null   int64 
 9   cl_text      4243 non-null   object
 10  text_len     4243 non-null   int64 
 11  rubric       4243 non-null   object
 12  year         4243 non-null   int64 
 13  lems         4243 non-null   object
 14  tit_1        4243 non-null   object
 15  tit_2        4243 non-null   object
dtypes: int64(3), object(13)
memory usage: 530.5+ KB


In [19]:
df['tsne_1'] = techs_vec_tsne[:,0]
df['tsne_2'] = techs_vec_tsne[:,1]

In [24]:
df.loc[df['author'] =="Дмитрий Карасев"]

,url,author,publ_date,title,subtitle,description,text,month_year,title_len,cl_text,text_len,rubric,year,lems,tit_1,tit_2,label,cl,tsne_1,tsne_2
376,https://gorky.media/context/krajne-pravye-kons...,Дмитрий Карасев,2022-02-08,«Крайне правые конспирологи — единственная сил...,Интервью с историческим социологом Майклом Манном,"Майкл Манн, живой классик исторической макросо...",Дмитрий Карасев: Как вы пережили период пандем...,2022-02,9,Дмитрий Карасев: Как вы пережили период пандем...,2544,context,2022,"['дмитрий', 'карасев', 'пережить', 'период', '...",«Крайне правые конспирологи —... 2022-02-08,Карасев Д. 2022-02-08,9,"Ист соц, соц",14.816406,-16.914579
3145,https://gorky.media/reviews/pochemu-indiya-ne-...,Дмитрий Карасев,2021-02-24,Почему Индия не стала Южной Кореей,Дмитрий Карасев — о книге экономиста Вивека Чи...,Во второй половине прошлого века Южная Корея с...,Догоняющее развитие связано с государственным ...,2021-02,6,Догоняющее развитие связано с государственным ...,1448,reviews,2021,"['догонять', 'развитие', 'связать', 'государст...",Почему Индия не стала... 2021-02-24,Карасев Д. 2021-02-24,7,"Отчеч история, WWI, 1917",2.668029,-24.767929
3548,https://gorky.media/reviews/fanatiki-protiv-sk...,Дмитрий Карасев,2019-12-26,Фанатики против скряг: что общего у средневеко...,О книге американского социолога Чарльза Тилли ...,Зачем в средневековой Франции на свадьбах вдов...,Чарльз Тилли. От мобилизации к революции. М.: ...,2019-12,11,Чарльз Тилли. От мобилизации к революции. М.: ...,2215,reviews,2019,"['чарльз', 'тилли', 'мобилизация', 'революция'...",Фанатики против скряг: что... 2019-12-26,Карасев Д. 2019-12-26,9,"Ист соц, соц",16.581726,-17.183968
3912,https://gorky.media/reviews/sotsialnoe-gosudar...,Дмитрий Карасев,2018-09-19,Социальное государство мучеников,Рецензия на книгу об Исламской революции в Ира...,Откуда в Иране взялось государство всеобщего б...,Kevan Harris. A Social Revolution: Politics an...,2018-09,3,Kevan Harris. A Social Revolution: Politics an...,4368,reviews,2018,"['социальный', 'революция', 'политик', 'госуда...",Социальное государство мучеников 2018-09-19,Карасев Д. 2018-09-19,9,"Ист соц, соц",11.879491,-20.588978


## Bokeh

In [9]:
# Обучил лучшую модель, с 11-ю кластерами
best_model = KMeans(n_clusters=11,
                    init="k-means++",
                    random_state=12345).fit(features)

In [10]:
# метки кластеров в список
labels = [str(x) for x in best_model.labels_]
# Добавил метки кластеров в датафрейм с названиями технологий
df['label'] = labels

In [11]:
#сжал 1024 до 2-х с помощью TSNE, чтобы изобразить точки кластеров в двухмерном пространстве
tsne = TSNE(n_components=2, random_state=12345)
techs_vec_tsne = tsne.fit_transform(features)

In [12]:
# список заголовков для графика
tt = list(df['tit_2'])


In [13]:
# создам столбец который заполню названиями кластеров
df['cl'] = 'e'

In [14]:
df.loc[df['label']=='0', 'cl'] = 'Короткие тексты'
df.loc[df['label']=='1', 'cl'] = 'Обзоры зарубеж книж инт Оборина'
df.loc[df['label']=='2', 'cl'] = 'Естественные науки'
df.loc[df['label']=='3', 'cl'] = 'Русская проза, лит премии'
df.loc[df['label']=='4', 'cl'] = 'ЖЗЛ, СССР и XIX в.'
df.loc[df['label']=='5', 'cl'] = 'Переводные книги, Азия'
df.loc[df['label']=='6', 'cl'] = 'Опасные темы'
df.loc[df['label']=='7', 'cl'] = 'Отчеч история, WWI, 1917'
df.loc[df['label']=='8', 'cl'] = 'ЖЗЛ, биогр., '
df.loc[df['label']=='9', 'cl'] = 'Ист соц, соц'
df.loc[df['label']=='10', 'cl'] = 'Фантастика, прикл,'

# список названий кластеров
cl = list(df.cl)

In [23]:
# количество кластеров должно соответствовать количеству меток здесь 6

output_notebook()

SPECIES = sorted(df.cl.unique())
MARKERS = ['plus', 'circle', 'triangle', 'square', 'star',
           'diamond', 'hex', 'triangle_pin', 'square_pin', 'asterisk',
           'inverted_triangle']

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Кластеры текстов Горького",
           background_fill_color="#fafafa",
           width=1500, height=1500)

p.xaxis.axis_label = 'TSNE_x'
p.yaxis.axis_label = 'TSNE_y'

source = ColumnDataSource(data=dict(x1=techs_vec_tsne[:,0],
                                    x2=techs_vec_tsne[:,1],
                                    names=tt,
                                    species=cl))

p.scatter(x="x1", y="x2", size=7, source=source,
          legend_group="species", fill_alpha=0.4,
          marker=factor_mark('species', MARKERS, SPECIES),
          color=factor_cmap('species', 'Category10_10', SPECIES))

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="6pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

p.legend.location = "top_left"
p.legend.title = "Clusters"

show(p)

## Plotly 3-D

In [25]:
pca = PCA(n_components=512, random_state=42)
techs_vec_pca = pca.fit_transform(features)

In [54]:
clustering = AgglomerativeClustering(n_clusters=None,
                                     distance_threshold=0.92, # threshold
                                     metric='cosine',
                                     linkage='average').fit(techs_vec_pca) # обучаемя на techs_vec_pca - сж до 512 эмбб YaGPT

In [55]:
pca = PCA(n_components=3, random_state=42)
emb_3d = pd.DataFrame(pca.fit_transform(features), columns=['x1', 'x2', 'x3']) # обучаемя на techs_vec_pca - сж до 64 эмбб
emb_3d['label'] = clustering.labels_
emb_3d['tit_2'] = df['tit_2']
emb_3d['subtitle'] = df['subtitle']


In [53]:
fig = px.scatter_3d(emb_3d,
                    x='x1', y='x2', z='x3',
                    color='label',
                    width=1200, height=1000,
                    hover_name = 'tit_2',
                    hover_data = 'subtitle',
                    opacity=1)
fig.show()